In [5]:


import numpy as np 
import cv2 

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

# In[6]:


img = cv2.imread("test2.png")
#imgResize = cv2.resize(image,(500,500))
imgCpy = cv2.imread("test2.png")
#==========================================================
#gray
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
imgGray = cv2.GaussianBlur(imgGray,(7,7),1)
ret,imgThresh=cv2.threshold(imgGray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#canny
kernel = np.ones((5, 5), np.uint8)
imgCanny = cv2.Canny(imgThresh,80,80)
imgMorph=cv2.dilate(imgCanny,kernel,iterations=1)

#All contours
contours, heirarchy = cv2.findContours(imgMorph,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
row,col=imgMorph.shape
maxCnt=contours[0]
maxArea=0
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
for i, cnt in enumerate(contours):
    image = cv2.polylines(imgCpy, cnt,True,(0,0,255),5)
    perimeter = cv2.arcLength(cnt,True)
    corners = cv2.approxPolyDP(cnt, 0.01*perimeter,True)
    #=================================
    area=cv2.contourArea(cnt)
    if((area>maxArea) and (area<row*col*0.95)):
        maxArea=area
        maxCnt=cnt
    #=================================
#     print("Number of Corners in this contour: ")
#     print(len(corners))
#     print("#==========================")
    #00000000000000000000000000000000000000000000000000000000000000000000000000
    #round rect over every contour
    contours_poly[i] = cv2.approxPolyDP(cnt, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
    #00000000000000000000000000000000000000000000000000000000000000000000000000

#maxCountour
cornersofMAX = cv2.approxPolyDP(cnt, 0.01*perimeter,True)
print(len(cornersofMAX))
#change of function
if(len(cornersofMAX)==4):
    #image is of closed maze
    print("else")
#======================================================================================================
drawing = np.zeros((imgCpy.shape[0],imgCpy.shape[1], 3), dtype=np.uint8)
for i in range(len(contours)):
    color = (0,255,0)
    colorp = (0,0,255)
    cv2.drawContours(drawing, contours_poly, i, colorp)
    cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])),(int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
print(boundRect)
boundRect = np.array(boundRect)
print(boundRect.shape)

#x y w h
#x y
#x y+h
#x+w , y+h
#x+w y
#===============================================
print(boundRect)
#allVertex = np.zeros((boundRect.shape[0],4,2),dtype = np.uint8) // 3 4 2
allVertex = list()
#print(allVertex.shape)

for i in range(0,boundRect.shape[0]):
    allVertex+=             [[
                            [boundRect[i][0], boundRect[i][1]],
                            [boundRect[i][0], boundRect[i][1]+boundRect[i][3]],
                            [boundRect[i][0]+boundRect[i][2], boundRect[i][1]+boundRect[i][3]],
                            [boundRect[i][0]+boundRect[i][2], boundRect[i][1]]
                            ]]
allVertex = np.array(allVertex)
print(allVertex)
top_left = [0,0]
top_right = [0,0]
bottom_left = [0,0]
bottom_right = [0,0]

print(allVertex[0][0][0])
print("#=")
#for top_left
minSum = 1e5
for i in range(0,allVertex.shape[0]):
    sumCordi = allVertex[i][0][0] + allVertex[i][0][1]
    if minSum>sumCordi:
        minSum = sumCordi
        top_left = [allVertex[i][0][0] ,allVertex[i][0][1]]
        
#for bottom_right
maxSum = 0
for i in range(0,allVertex.shape[0]):
    sumCordi = allVertex[i][2][0] + allVertex[i][2][1]
    if minSum<sumCordi:
        minSum = sumCordi
        bottom_right = [allVertex[i][2][0], allVertex[i][2][1]]
        
print("topright all")
#for top_right(diff max-->(x-y))
maxdiff = 0
for i in range(0,allVertex.shape[0]):
    diffCordi = allVertex[i][3][0] - allVertex[i][3][1]
    print(allVertex[i][3][0],allVertex[i][3][1])
    if maxdiff<diffCordi:
        maxdiff = diffCordi
        top_right = [allVertex[i][3][0], allVertex[i][3][1]]

#for bottom_left(diff max-->(y-x))
maxdiff = 0
for i in range(0,allVertex.shape[0]):
    diffCordi = allVertex[i][1][1] - allVertex[i][1][0]
    if maxdiff<diffCordi:
        maxdiff = diffCordi
        bottom_left = [allVertex[i][1][0], allVertex[i][1][1]]
        
drawing = cv2.circle(drawing, tuple(top_left), 20, (255,0,0), 6) #correct
drawing = cv2.circle(drawing, tuple(bottom_right), 20, (255,0,0), 6) #correct
# print("topright and fucking bottom left")
# print(top_right)
# print(bottom_left)
drawing = cv2.circle(drawing, tuple(top_right), 20, (255,0,0), 6)
drawing = cv2.circle(drawing, tuple(bottom_left), 20, (255,0,0), 6)
#=============================================
Height = bottom_left[1] - top_left[1]
Width = top_right[0] - top_left[0]
#=======================================================
# rect = np.zeros((4, 2), dtype="float32")
# # the top-left point will have the smallest sum, whereas
# # the bottom-right point will have the largest sum

# s = allVertex.sum(axis=2)

# rect[0] = allVertex[np.argmin(s)]
# rect[2] = allVertex[np.argmax(s)]

#     #print(s)
#     #print(rect[0],rect[2])

#     # now, compute the difference between the points, the
#     # top-right point will have the smallest difference,
#     # whereas the bottom-left will have the largest difference
    
# diff = np.diff(allVertexl, axis=2)
#     #print(diff)

# rect[1] = allVertex[np.argmin(diff)]
# rect[3] = allVertex[np.argmax(diff)]
#=======================================================



dst = np.array([
        [0, 0],
        [Width+1, 0],
        [Width+1, Height+1],
        [0, Height+1]], dtype = "float32")
rect = np.zeros((4, 2), dtype="float32")
print([[top_left[0],top_left[1]], [top_right[0],top_right[1]], [bottom_right[0],bottom_right[1]], [bottom_left[0],bottom_left[1]]])

rect = [[top_left[0],top_left[1]], [top_right[0],top_right[1]], [bottom_right[0],bottom_right[1]], [bottom_left[0],bottom_left[1]]]

rect = np.array(rect,np.float32)
print(rect.shape)
M = cv2.getPerspectiveTransform(rect, dst)
warped_img = cv2.warpPerspective(img, M, (Width, Height))
warped_img = cv2.resize(warped_img, (1280, 1280))




#======================================================================================================
stack = stackImages(0.35,([img,imgGray,imgThresh],[imgCanny,drawing,warped_img]))

cv2.imshow("images",stack)
cv2.waitKey(0)
cv2.destroyAllWindows()



24
[(124, 145, 118, 97), (13, 124, 118, 118), (38, 102, 93, 9), (13, 13, 229, 140)]
(4, 4)
[[124 145 118  97]
 [ 13 124 118 118]
 [ 38 102  93   9]
 [ 13  13 229 140]]
[[[124 145]
  [124 242]
  [242 242]
  [242 145]]

 [[ 13 124]
  [ 13 242]
  [131 242]
  [131 124]]

 [[ 38 102]
  [ 38 111]
  [131 111]
  [131 102]]

 [[ 13  13]
  [ 13 153]
  [242 153]
  [242  13]]]
124
#=
topright all
242 145
131 124
131 102
242 13
[[13, 13], [242, 13], [242, 242], [13, 242]]
(4, 2)
